In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install recommenders
!pip install papermill
!pip install tensorflow pandas
!pip install pandera==0.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 8.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.0/356.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.6/259.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.sparse import csr_matrix

In [4]:
import os
import sys
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from google.colab import files
from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF # preprocessing steps - reconfigure for additional column names/data types
from recommenders.models.ncf.dataset import Dataset as NCFDataset
# from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_chrono_split # splits into train/test
from recommenders.evaluation.python_evaluation import (
    map, ndcg_at_k, precision_at_k, recall_at_k
)
from recommenders.utils.constants import SEED as DEFAULT_SEED
from recommenders.utils.notebook_utils import store_metadata

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]
Pandas version: 2.2.2
Tensorflow version: 2.17.0


In [5]:
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
# MOVIELENS_DATA_SIZE = '100k'

# Model parameters
EPOCHS = 1
BATCH_SIZE = 256

SEED = DEFAULT_SEED  # Set None for non-deterministic results

In [6]:
df = pd.read_csv('/content/drive/MyDrive/NCF/ml-1m_dataset.csv')

In [7]:
df.head()

,userID,itemID,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
train, test = python_chrono_split(df, 0.75)

In [9]:
test = test[test["userID"].isin(train["userID"].unique())]
test = test[test["itemID"].isin(train["itemID"].unique())]

In [10]:
leave_one_out_test = test.groupby("userID").last().reset_index()

In [11]:
train_file = "./1Mtrain.csv"
test_file = "./1Mtest.csv"
leave_one_out_test_file = "./1Mleave_one_out_test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)
leave_one_out_test.to_csv(leave_one_out_test_file, index=False)

In [12]:
# train_data = pd.read_csv('/content/drive/MyDrive/NCF/1Mtrain.csv')
# test_data = pd.read_csv('/content/drive/MyDrive/NCF/1Mtest.csv')
# leave_one_out_test_data = pd.read_csv('/content/drive/MyDrive/NCF/1Mleave_one_out_test.csv')


In [13]:
# # Save updated files back to Google Drive
# train_file.to_csv('/content/drive/MyDrive/1Mtrain.csv', index=False)
# test_file.to_csv('/content/drive/MyDrive/1Mtest.csv', index=False)
# leave_one_out_test_file.to_csv('/content/drive/MyDrive/1Mleave_one_out_test.csv', index=False)

In [14]:
data = NCFDataset(
    train_file=train_file,
    test_file=test_file,
    seed=SEED,
    overwrite_test_file_full=False  # Set to False to use existing files without recreating
)

Streaming output truncated to the last 5000 lines.
 59%|█████▊    | 3541/6040 [05:13<04:18,  9.65it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
 59%|█████▊    | 3544/6040 [05:13<03:14, 12.80it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a fut

In [15]:
# import joblib

# # Save the data object to Google Drive
# joblib.dump(data, '/content/drive/MyDrive/ncf_dataset_data.joblib')

In [16]:

# !cp ./train.csv /content/drive/MyDrive/train.csv
# !cp ./test.csv /content/drive/MyDrive/test.csv

# files.download('./train.csv')
# files.download('./test.csv')
# files.download('./leave_one_out_test.csv')

In [17]:
model = NCF(
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [18]:
# from time import time, sleep
# from tqdm import tqdm
# from threading import Thread

# # Estimated training time in seconds (adjust as needed)
# estimated_training_duration = 60

# # Function to display progress bar while model is training
# def show_progress_bar():
#     for _ in tqdm(range(100), desc="Training Progress", unit="%"):
#         sleep(estimated_training_duration / 100)

# # Start the progress bar in a separate thread
# progress_thread = Thread(target=show_progress_bar)
# progress_thread.start()

# with Timer() as train_time:
#     # Run model fitting in the main thread
#     model.fit(data)

# # Ensure progress bar completes once training is done
# progress_thread.join()
# print("Took {} seconds for training.".format(train_time.interval))


In [19]:
import time
from tensorflow.keras.callbacks import Callback

class TrainingProgressCallback(Callback):
    def on_train_begin(self, logs=None):
        self.start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        progress = (epoch + 1) / self.params['epochs'] * 100
        elapsed_time = time.time() - self.start_time
        print(f"Epoch {epoch + 1}/{self.params['epochs']} - {progress:.2f}% complete - Elapsed time: {elapsed_time:.2f} seconds")

In [23]:
from time import time

total_epochs = EPOCHS

start_time = time()
for epoch in range(total_epochs):
    model.fit(data)
    progress = (epoch + 1) / total_epochs * 100
    elapsed_time = time() - start_time
    print(f"Epoch {epoch + 1}/{total_epochs} - {progress:.2f}% complete - Elapsed time: {elapsed_time:.2f} seconds")
end_time = time()
print("Took {} seconds for training.".format(end_time - start_time))

Epoch 1/10 - 10.00% complete - Elapsed time: 1161.19 seconds
Epoch 2/10 - 20.00% complete - Elapsed time: 2350.83 seconds
Epoch 3/10 - 30.00% complete - Elapsed time: 3495.90 seconds
Epoch 4/10 - 40.00% complete - Elapsed time: 4743.50 seconds
Epoch 5/10 - 50.00% complete - Elapsed time: 6002.09 seconds
Epoch 6/10 - 60.00% complete - Elapsed time: 7256.19 seconds
Epoch 7/10 - 70.00% complete - Elapsed time: 8502.55 seconds
Epoch 8/10 - 80.00% complete - Elapsed time: 9681.68 seconds
Epoch 9/10 - 90.00% complete - Elapsed time: 10816.97 seconds
Epoch 10/10 - 100.00% complete - Elapsed time: 11978.37 seconds
Took 11978.371688365936 seconds for training.


In [24]:
# with Timer() as train_time:
#     model.fit(data)

# print("Took {} seconds for training.".format(train_time.interval))

In [3]:
predictions = [[row.userID, row.itemID, model.predict(row.userID, row.itemID)]
               for (_, row) in test.iterrows()]


predictions = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])
predictions.head()

NameError: name 'test' is not defined

In [26]:
with Timer() as test_time:

    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time.interval))

Took 172.38446967799973 seconds for prediction.


In [1]:
eval_map = map(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

NameError: name 'test' is not defined

In [2]:
model.save('/content/drive/MyDrive/NCF/model')

NameError: name 'model' is not defined

In [ ]:
# from tensorflow.keras.models import load_model
# model = load_model('/path/to/save/your_model')